# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [99]:
data_set=None
labels=None
test_data_set=None
test_labels=None
unique_labels=None
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [100]:
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [101]:
def build_classifiers():
    classifiers = [classifier() for classifier in  [ LinearRegression, KNeighborsClassifier, SVC, DecisionTreeClassifier, GaussianNB, QuadraticDiscriminantAnalysis, ]]
    for classifier in classifiers:
        classifier.fit(data_set, labels)
    return classifiers # and here

In [102]:
def build_stacked_classifier(classifiers):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    output = np.array(output).reshape((130,3))
    
    # stacked classifier part:
    stacked_classifier = DecisionTreeClassifier() # set here
    stacked_classifier.fit(output.reshape((130,3)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]),3))
    predicted = stacked_classifier.predict(test_set)
    return predicted

In [103]:
from operator import itemgetter
from itertools import permutations

classifiers = build_classifiers()
returned = []
for permutation in permutations(classifiers, r=3):
    predicted = build_stacked_classifier(permutation)
    accuracy = accuracy_score(test_labels, predicted)
    returned.append((accuracy, '\t>\t'.join([f'{str(x)[:-2]:30}' for x in permutation])))
returned.sort(key=itemgetter(0), reverse=True)
for i, max_res in enumerate(returned[:10]):
    print(f'{i+1})\t{max_res[0]:1.2f} \t->\t {max_res[1]}')

1)	0.95 	->	 DecisionTreeClassifier        	>	KNeighborsClassifier          	>	QuadraticDiscriminantAnalysis 
2)	0.90 	->	 KNeighborsClassifier          	>	DecisionTreeClassifier        	>	QuadraticDiscriminantAnalysis 
3)	0.90 	->	 KNeighborsClassifier          	>	QuadraticDiscriminantAnalysis 	>	DecisionTreeClassifier        
4)	0.90 	->	 GaussianNB                    	>	KNeighborsClassifier          	>	DecisionTreeClassifier        
5)	0.90 	->	 GaussianNB                    	>	KNeighborsClassifier          	>	QuadraticDiscriminantAnalysis 
6)	0.90 	->	 QuadraticDiscriminantAnalysis 	>	KNeighborsClassifier          	>	DecisionTreeClassifier        
7)	0.85 	->	 KNeighborsClassifier          	>	SVC                           	>	DecisionTreeClassifier        
8)	0.85 	->	 KNeighborsClassifier          	>	SVC                           	>	GaussianNB                    
9)	0.85 	->	 KNeighborsClassifier          	>	SVC                           	>	QuadraticDiscriminantAnalysis 
10)	0.85 	

## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [104]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

def calculate_error(model):
    predicted = model.predict(test_set)
    I=calculate_accuracy_vector(predicted, test_labels)
    Z=np.sum(I)
    return (1+Z)/1.0

def calculate_accuracy_vector(predicted, test_labels):
    returned = []
    for index in range(len(predicted)):
        if predicted[index] == test_labels[index]:
            returned.append(0)
        else:
            returned.append(1)
    return returned

Fill the two functions below:

In [105]:
def set_new_weights(model):
    result = np.array(calculate_accuracy_vector(model.predict(test_set), test_labels))
    return (result + 1) / result.sum()

Train the classifier with the code below:

In [106]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

print(weights)


validate_x, validate_label = generate_data(1, dimension, labels)

[0.00196078 0.00196078 0.00392157 0.00196078 0.00196078 0.00392157
 0.00196078 0.00392157 0.00196078 0.00392157 0.00196078 0.00196078
 0.00392157 0.00392157 0.00392157 0.00392157 0.00392157 0.00392157
 0.00196078 0.00196078 0.00392157 0.00196078 0.00392157 0.00392157
 0.00392157 0.00196078 0.00196078 0.00196078 0.00196078 0.00196078
 0.00196078 0.00392157 0.00392157 0.00196078 0.00392157 0.00196078
 0.00196078 0.00196078 0.00196078 0.00392157 0.00392157 0.00196078
 0.00392157 0.00196078 0.00392157 0.00196078 0.00196078 0.00392157
 0.00392157 0.00392157 0.00392157 0.00392157 0.00392157 0.00196078
 0.00196078 0.00196078 0.00196078 0.00196078 0.00392157 0.00196078
 0.00392157 0.00392157 0.00196078 0.00392157 0.00196078 0.00392157
 0.00392157 0.00392157 0.00196078 0.00196078 0.00196078 0.00196078
 0.00392157 0.00196078 0.00392157 0.00392157 0.00392157 0.00196078
 0.00196078 0.00196078 0.00196078 0.00392157 0.00196078 0.00392157
 0.00196078 0.00392157 0.00392157 0.00392157 0.00196078 0.0019

Set the validation data set:

In [107]:
validate_x, validate_label = generate_data(1, dimension, labels)

Fill the prediction code:

In [108]:
def get_prediction(x):
    predictions = [classifier.predict(x) for classifier in classifiers]
    scores = [0, 0]
    for prediction, weight in enumerate(predictions):
        if prediction == 0:
            scores[0] += weights[weight]
        else:
            scores[1] += weights[weight]
    if scores[0] < scores[1]:
        return 1
    else:
        return 0

Test it:

In [109]:
prediction = get_prediction(validate_x) #[0]
print(prediction)

1
